In [2]:
from sqlalchemy import create_engine
# Set Up Environment
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize


# Database credentials
postgres_user = 'postgres'
postgres_pw = 'postgres'
postgres_host = 'localhost'
postgres_port = '5432'
postgres_db = 'bev_rec'

# Use the credentials to start a connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

In [9]:
# Initial sql query, stored in sql variable
sql = '''
  SELECT *
  FROM public.test;
'''

results = engine.execute(sql)
engine.dispose()
rows = results.fetchall()




# print some results just to see what we got
for row in rows:
  print(row)

(1, 'Michael')


In [3]:
# List of parameters to search DB
search_parameters = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

url = 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f='
json_list = []

for i in search_parameters:
    r = requests.get(url+str(i))
    # This presumes your JSON response is a dict, if the response is a list, use extend instead of append
    json_list.append(r.json())
    
df = pd.json_normalize(json_list, 'drinks')

In [46]:
df_new = df[['idDrink', 'strDrink', 'strCategory', 'strIBA', 'strAlcoholic', 
         'strGlass', 'strInstructions', 'strDrinkThumb', 'strIngredient1', 
         'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5', 
         'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9', 
         'strIngredient10', 'strMeasure1', 'strMeasure2', 'strMeasure3', 'strMeasure4', 
         'strMeasure5', 'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10']]

df_new['all_ingredients'] = df_new[df_new.columns[8:17]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   idDrink          423 non-null    object
 1   strDrink         423 non-null    object
 2   strCategory      423 non-null    object
 3   strIBA           63 non-null     object
 4   strAlcoholic     423 non-null    object
 5   strGlass         423 non-null    object
 6   strInstructions  423 non-null    object
 7   strDrinkThumb    423 non-null    object
 8   strIngredient1   423 non-null    object
 9   strIngredient2   423 non-null    object
 10  strIngredient3   356 non-null    object
 11  strIngredient4   226 non-null    object
 12  strIngredient5   138 non-null    object
 13  strIngredient6   65 non-null     object
 14  strIngredient7   28 non-null     object
 15  strIngredient8   7 non-null      object
 16  strIngredient9   3 non-null      object
 17  strIngredient10  1 non-null      ob

<ipython-input-46-99003b3c1c6f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['all_ingredients'] = df_new[df_new.columns[8:17]].apply(


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
#df_new.loc[:('strInstructions')] = df_new.loc[:('strInstructions')].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_new['all_ingredients'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(423, 280)

In [40]:
tfidf.get_feature_names()[50:65]

['caramel',
 'carbonated',
 'cardamom',
 'cassis',
 'cayenne',
 'celery',
 'chambord',
 'champagne',
 'chartreuse',
 'cherries',
 'cherry',
 'chili',
 'chocolate',
 'cider',
 'cinnamon']

In [41]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

cosine_sim.shape

(423, 423)

In [42]:
#Construct a reverse map of indices and movie titles
#indices = pd.Series(df_new.index, index=df_new['idDrink']).drop_duplicates()
indices = pd.Series(df_new.index, index=df_new['strDrink']).drop_duplicates()
indices[:10]

strDrink
155 Belmont                    0
1-900-FUK-MEUP                 1
110 in the shade               2
151 Florida Bushwacker         3
252                            4
24k nightmare                  5
3 Wise Men                     6
3-Mile Long Island Iced Tea    7
410 Gone                       8
50/50                          9
dtype: int64

In [47]:
# Function that takes in drink name as input and outputs most similar drinks
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the drink indices
    drink_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df_new[['strDrink', 'all_ingredients']].iloc[drink_indices]

In [49]:
get_recommendations('B-53')

,strDrink,all_ingredients
41,B-52,"Baileys irish cream,Grand Marnier,Kahlua"
371,Talos Coffee,"Grand Marnier,Coffee"
15,A1,"Gin,Grand Marnier,Lemon Juice,Grenadine"
9,50/50,"Vanilla vodka,Grand Marnier,Orange juice"
98,Dirty Nipple,"Kahlua,Sambuca,Baileys irish cream"
269,New York Lemonade,"Absolut Citron,Grand Marnier,Lemon juice,Club ..."
136,Flaming Lamborghini,"Kahlua,Sambuca,Blue Curacao,Baileys irish cream"
123,Freddy Kruger,"Jägermeister,Sambuca,Vodka"
189,Iced Coffee Fillip,"Kahlua,Coffee"
135,Flander's Flake-Out,"Sambuca,Sarsaparilla"
